# Variable Shapes

In [2]:
import d2ltvm
import numpy as np
import tvm
from tvm import te
# te.var !!!
n = te.var(name='n')
type(n), n.dtype
print(n)

successd...

n: int32


In [13]:
def vector_add_vs(n, dtype):
    A = te.placeholder((n,), name='a', dtype=dtype)
    B = te.placeholder((n,), name='b', dtype=dtype)
    C = te.compute(A.shape, lambda i: A[i] + B[i])
    s = te.create_schedule(C.op)
    return s, A, B, C

s, A, B, C = vector_add_vs(n, 'float32')
print(tvm.lower(s, [A, B, C], simple_mode=True))

@main = primfn(a_1: handle, b_1: handle, compute_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {a: Buffer(a_2: Pointer(float32), float32, [(stride: int32*n: int32)], [], type="auto"),
             b: Buffer(b_2: Pointer(float32), float32, [(stride_1: int32*n)], [], type="auto"),
             compute: Buffer(compute_2: Pointer(float32), float32, [(stride_2: int32*n)], [], type="auto")}
  buffer_map = {a_1: a, b_1: b, compute_1: compute}
  preflattened_buffer_map = {a_1: a_3: Buffer(a_2, float32, [n], [stride], type="auto"), b_1: b_3: Buffer(b_2, float32, [n], [stride_1], type="auto"), compute_1: compute_3: Buffer(compute_2, float32, [n], [stride_2], type="auto")} {
  for (i: int32, 0, n) {
    compute[(i*stride_2)] = (a[(i*stride)] + b[(i*stride_1)])
  }
}




In [16]:
def test_mod(mod, n, dtype):
    '''n const value
    '''
    a, b, c = d2ltvm.get_abc(n, lambda x:tvm.nd.array(x.astype(dtype)))
    mod(a, b, c)
    print("c.shape:", c.shape)
    np.testing.assert_equal(c.asnumpy(), a.asnumpy() + b.asnumpy())
    
mod = tvm.build(s, [A, B, C])
test_mod(mod, 100, 'float32')

c.shape: (100,)


# Multi-dimensional Shapes 

In [27]:
def tvm_vector_add(ndims):
    '''define input(placeholder) ---> define op(te.compute) ---> create_schedule ---> build(compiler to machine code)
    '''
    A = te.placeholder([te.var() for _ in range(ndims)], name='a')
    B = te.placeholder(A.shape, name='b')
    C = te.compute(A.shape, lambda *i: A[i] + B[i], name='c')
    s = te.create_schedule(C.op)
    return s, A, B, C


In [30]:
s, A, B, C = tvm_vector_add(2)
mod = tvm.build(s, [A, B, C])
test_mod(mod, (3,2), 'float32')

s, A, B, C = tvm_vector_add(4)
mod = tvm.build(s, [A, B, C])
test_mod(mod,(2,3,4,5), 'float32')

c.shape: (3, 2)
c.shape: (2, 3, 4, 5)


# Summary
1.We can use te.var() to specify the dimension(s) of a shape when we don’t know the concrete data shape before execution

2.The shape of an n-dimensional tensor is presented as an n-length tuple